# Signal Filter Design for JETI

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.optimize import minimize
from scipy.optimize import Bounds

In [ ]:
def double_exponential_filter(altitude,
                              old_altitude_1,
                              old_altitude_2,
                              old_climb_rate,
                              tau_1, tau_2, dyn_alpha_divisor,
                              delta_t):

        alfa_1 = delta_t / (tau_1 + delta_t)
        alfa_2 = delta_t / (tau_2 + delta_t)

        smoothed_altitude_1 = old_altitude_1 - alfa_1 * (old_altitude_1 - altitude)
        smoothed_altitude_2 = old_altitude_2 - alfa_2 * (old_altitude_2 - altitude)

        # calc default gain from time constants chosen
        factor = 1 / (tau_2 - tau_1 + 1.e-9)

        climb_rate = (smoothed_altitude_1 - smoothed_altitude_2) * factor

        dyn_alpha = abs((old_climb_rate - climb_rate) / dyn_alpha_divisor)
        dyn_alpha = min(dyn_alpha, 1.0)
        smoothed_climb_rate = old_climb_rate - dyn_alpha * (old_climb_rate - climb_rate)

        return smoothed_altitude_1, smoothed_altitude_2, smoothed_climb_rate
